<a href="https://colab.research.google.com/github/panditamey1/nlp_refresher/blob/main/nlp_refresher_7_fine_tune_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [25]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

In [6]:
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model=BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [ ]:
model

In [10]:
model = model.to("cuda")

In [11]:
sample_data = ["I am eating", "I am playing"]

tokenizer(sample_data, padding=True, truncation=True, max_length = 512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [12]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/jigsaw-toxic-comment-train.csv")

In [13]:
data = data[:1000]
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [16]:
X = list(data['comment_text'])
y = list(data['toxic'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [17]:
X_train_tokenize = tokenizer(X_train, padding=True, truncation=True, max_length = 512)
X_test_tokenize = tokenizer(X_test, padding=True, truncation=True, max_length=512)


In [18]:
X_train_tokenize.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
X_train_tokenize['input_ids'][0]

In [20]:
len(X_train)

800

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files

import torch

class IMDbDataset(torch.utils.data.Dataset):
    
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [21]:
import torch
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
      self.encodings = encodings
      self.labels = labels

  def __getitem__(self, idx):
      item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
      item['labels'] = torch.tensor(self.labels[idx])
      return item

  def __len__(self):
      return len(self.encodings['input_ids'])

In [22]:
train_dataset = Dataset(X_train_tokenize, y_train)
test_dataset = Dataset(X_test_tokenize, y_test)

In [23]:
train_dataset[5]


{'input_ids': tensor([  101,  1000,  2043,  1045,  2246,  2012,  1996,  4937,  2080, 10086,
          1010,  2009,  2001,  3432,  1037,  8208,  1997,  1996,  4431,  1010,
          1996,  2804,  1997,  2178, 15009,  1010,  1998,  1037,  2689,  1999,
          1996,  2653,  1012,  2302,  2019, 10086, 12654,  2000,  4863,  1010,
          1996,  3431,  2020,  4895, 29427,  7810,  1012,  1006,  2115, 18856,
          8486, 10803,  5121,  7126,  1012,  1007,  4283,  1010,  2036,  1010,
          2005,  5815,  1996,  4431,  2006, 10945,  1012, 19597,  1010,  1045,
          2106,  2025,  2298,  2012,  1996,  4216,  3718,  2030,  2794,  1012,
          2026,  2364,  5142,  2001,  1996, 16655,  2595, 24759, 18175,  2094,
         10086,  2015,  1012,  7680,  7849,  3111,  2024,  2590,   999,  1998,
          2059,  1045,  2387,  2008,  1996,  7696,  2055,  2122,  2168,  5936,
          2013,  1037,  2096,  2067,  1010,  2061,  2026,  4471,  2003,  2062,
          1997,  1037, 14764,  1012,  2

In [30]:
def compute_metrics(p):
  print(type(p))
  pred, labels = p

  pred = np.argmax(pred, axis = 1)

  accuracy = accuracy_score(y_true=labels, y_pred = pred)
  recall = recall_score(y_true=labels, y_pred = pred)
  precision = precision_score(y_true=labels, y_pred = pred)
  f1 = f1_score(y_true=labels, y_pred = pred)

  return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1  }



In [31]:
args = TrainingArguments(
    output_dir='output',
    num_train_epochs=1,
    per_device_train_batch_size=8

)
# interface by huggingface takes model, arguments, train and validation dataset
trainer = Trainer(
    model = model,
    args = args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [32]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 800
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 100
  Number of trainable parameters = 109483778


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.278222599029541, metrics={'train_runtime': 77.9498, 'train_samples_per_second': 10.263, 'train_steps_per_second': 1.283, 'total_flos': 210488844288000.0, 'train_loss': 0.278222599029541, 'epoch': 1.0})

In [33]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.18867704272270203,
 'eval_accuracy': 0.95,
 'eval_precision': 0.8666666666666667,
 'eval_recall': 0.6190476190476191,
 'eval_f1': 0.7222222222222222,
 'eval_runtime': 6.669,
 'eval_samples_per_second': 29.99,
 'eval_steps_per_second': 3.749,
 'epoch': 1.0}

In [34]:
text = "That was a good point"

inputs = tokenizer(text, padding=True, truncation=True,return_tensors='pt').to('cuda')

In [35]:
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits,dim = -1)

predictions = predictions.cpu().detach().numpy()
predictions

array([[0.984622  , 0.01537799]], dtype=float32)

In [36]:
trainer.save_model('CustomModel')

Saving model checkpoint to CustomModel
Configuration saved in CustomModel/config.json
Model weights saved in CustomModel/pytorch_model.bin


In [ ]:
model_2 = BertForSequenceClassification.from_pretrained('CustomModel')
model_2.to('cuda')

In [39]:
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits,dim = -1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.3338, -1.8255]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9846, 0.0154]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.984622  , 0.01537799]], dtype=float32)